In [1]:
import langchain, langchain_postgres, langchain_openai
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI                   # or any chat-style LLM
from langchain_postgres import PGVector                
from langchain_core.tools import Tool
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain.agents.react.agent import create_react_agent
from langchain.agents import AgentExecutor


In [2]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

/var/folders/gm/qcj171q57c91jhxnv2qs65540000gn/T/ipykernel_9506/3951078898.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


In [3]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4.1-mini", model_provider="openai")

In [4]:
from sqlalchemy import create_engine
from langchain_postgres.vectorstores import PGVector

DATABASE_URL = os.environ["NEON_DB_URL"]
# Option A: pass the URL + engine_args
primary_store = PGVector(
    embeddings=embeddings,
    connection=DATABASE_URL,
    collection_name="nietzsche_primary",
    engine_args={"pool_pre_ping": True},
    use_jsonb=True,
    create_extension=True,
)


# ———————————————————————————————————————————————
# 2. SECONDARY STORE
# ———————————————————————————————————————————————

secondary_store = PGVector(
    embeddings=embeddings,
    connection=DATABASE_URL,
    collection_name="nietzsche_secondary",
    engine_args={"pool_pre_ping": True},
    use_jsonb=True,
    create_extension=True,
)



In [5]:
primary_retriever = primary_store.as_retriever(search_kwargs={"k":3})          # top-4 chunks
secondary_retriever = secondary_store.as_retriever(search_kwargs={"k":3})

In [6]:
from langchain_core.tools import Tool

primary_tool = Tool.from_function(
    func=lambda q: "\n\n".join(d.page_content for d in primary_retriever.get_relevant_documents(q)),
    name="primary_search",
    description=(
        "Search Nietzsche’s primary texts. "
        "Returns up to 3 most‐relevant chunks."
    ),
)

secondary_tool = Tool.from_function(
    func=lambda q: "\n\n".join(d.page_content for d in secondary_retriever.get_relevant_documents(q)),
    name="secondary_search",
    description=(
        "Search secondary analyses of Nietzsche. "
        "Returns up to 3 most‐relevant chunks."
    ),
)


In [7]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [8]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict, Annotated
from langgraph.graph.message import add_messages

class State(TypedDict):
    questions: Annotated[List[str], add_messages]
    primary_context: List[Document]
    secondary_context: List[Document]
    answer: str

In [9]:
from langchain_core.prompts import PromptTemplate
template = """
You are zarabot-ai, a retrieval-augmented chatbot specialized in Friedrich Nietzsche.
You are performing the first step of a multi-step process.
You have been given a new question or response in a conversation chain.
You need to decide if we need to retrieve information about Nietzche to answer it.
If it does not relate to Nietzche, just say "generate" nothing else.
If it does relate to Nietzche, just say "retrieve_secondary" nothing else.

Conversation History:
{conversation_history}

Most Recent Question:
{question}

Classification:
"""

# Build the PromptTemplate
classification_prompt = PromptTemplate.from_template(template)
def start_edge(state: State) -> str:
    question_history = "none"
    if len(state["questions"]) > 0:
        question_history = "\n".join(state["questions"][-1])
    messages = classification_prompt.invoke({"conversation_history": question_history,
                                   "question": state["question"]}).to_messages()
    return llm.invoke({"messages": messages}).content

In [11]:
template = """
You are zarabot-ai, a retrieval-augmented chatbot specialized in Friedrich Nietzsche.
TOOLS:
{tools}

TOOL NAMES:
{tool_names}

You are performing the second step of a multi-step process.
You have been given a question or response in a conversation chain.
You will search essays and books on Nietzsche to find information relevant to the user's question.

When reasoning, prefix your thoughts with "Thought:".
When calling a tool, use exactly:

Action: <tool_name>
Action Input: <query or JSON>

After the tool runs, the framework will inject:

Observation: <tool output>

Follow these numbered steps, but still use the ReAct tags:

1. Formulate a Thought describing what the user wants to know and the themes involved.
2. Issue an Action with the appropriate tool and Action Input to search for information.
3. Review the Observation and note whether it is useful; if it isn’t, explain why and return to step 1.
4. Once you have found sufficient information write and answer using the **Final Answer:** tag, provide the 3 most relevant 1-3 sentence snippents of the information you found these should be directly from the text.


INPUT:
{input}

CHAIN OF THOUGHT:
{agent_scratchpad}
"""


In [10]:
prompt = PromptTemplate.from_template(template)
def retrieve_secondary(state: State) -> State:
    agent = create_react_agent(
        model=llm,
        tools=[secondary_tool],
        prompt=prompt,
    )
    agent_executor = AgentExecutor(agent=agent, tools=[secondary_tool], verbose=True)
    return {**state, "secondary_context": agent_executor.invoke(state["question"].to_string())}


In [12]:
prompt = PromptTemplate.from_template(template)
agent = create_react_agent(llm, [secondary_tool], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[secondary_tool], verbose=True)


In [13]:
from langchain.agents.agent import OutputParserException

try:
    result = agent_executor.invoke({
        "input": "explain the start of Thus Spoke Zarathustra when he first leaves isolation",
        "agent_scratchpad": ""
    })
    print(result)
except OutputParserException as e:
    print("⏺ RAW LLM OUTPUT ⏺")
    print(e.bad_output)
    raise



> Entering new AgentExecutor chain...
Thought: The user wants an explanation of the beginning of Nietzsche's "Thus Spoke Zarathustra," focusing on the moment Zarathustra leaves his isolation. Key themes likely include Zarathustra's solitude, his motivation for leaving, and the philosophical significance of this departure, such as the transition from isolation to engagement with the world.

Action: secondary_search  
Action Input: "Thus Spoke Zarathustra beginning Zarathustra leaves isolation"

/var/folders/gm/qcj171q57c91jhxnv2qs65540000gn/T/ipykernel_9506/1527174260.py:13: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  func=lambda q: "\n\n".join(d.page_content for d in secondary_retriever.get_relevant_documents(q)),


ZARATHUSTRA'S SPEECHES: PART THREE 191
ing but the random play of chaos has produced the spider, but, once
produced, the spider is a necessary and purposeless event
Chance does not convey the sense of freedom, of an open space for
unpredictable alternatives. It designates instead the chaotic origins of
necessity. The invocation to create is accordingly a noble lie, as empty
of inner weight as is the heaven of gods before sunrise.
"On Passing By" (Section 7)
In Section 6 ("On the Mount of Olives"), Zarathustra explains his
loneliness as imposed by the distance between himself and human
beings; he is forced to conceal himself in order to avoid the crucifixion
that befell Christ. The entire description of his isolation and conceal-
ment is expressed in a long metaphor of winter and ice: an echo of the
Hyperboreans (218-21).
After visiting various peoples (Volk) and towns, Zarathustra sets out
for his mountaintop cave. He is taking detours rather than the main

ZARATHUSTRA'S SPEECHES: PART

In [15]:
result = agent_executor.invoke({
        "input": "what does the old man mean when he said Goeth he not along like a dancer to zarathustra?",
        "agent_scratchpad": ""
    })



> Entering new AgentExecutor chain...
Thought: The user wants to understand the meaning behind the old man's remark in Nietzsche's *Thus Spoke Zarathustra*, specifically the phrase "Goeth he not along like a dancer to Zarathustra?" This likely relates to the style, spirit, or approach Zarathustra embodies, perhaps contrasting the old man's perception with Zarathustra's demeanor. To clarify, I should look into Nietzsche's work or secondary analyses that explain this scene and its symbolic meaning.

Action: secondary_search  
Action Input: "Goeth he not along like a dancer to Zarathustra meaning old man thus spoke zarathustra"enlightenment. On his way down from the mountain where he has
been leading his solitary existence, Zarathustra comes across an old
man who recognizes him, but remarks how much he has changed
since he last saw him. The old man remarks that Zarathustra has
become like a child again, an 'awakened one' (like the Buddha), but
asks what he wants with the 'sleepers' belo